In [1]:
import json
import pandas as pd

In [2]:
# Opens the tweets file in read mode
with open('collected_tweets2020-12-07_13:29:12.txt','r') as file:
    tweets = file.readlines()

In [3]:
# Parse tweet to json format (list)
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [4]:
# Define a function that handles the tweet and converts it to a dataframe
def tweet_to_df(tweet):
    # Transform parsed_tweets into table format
    df_tratado = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]

    # Remove unnecessary columns
    df_tratado.drop(columns=[
    'quote_count'
    ,'reply_count'
    ,'retweet_count'
    ,'favorite_count'
    ,'favorited'
    ,'retweeted'
    ,'user'
    ,'entities'
    ,'retweeted_status'], inplace=True)

    # Select fields from the user object and add them to the Dataframe
    df_tratado['user_id'] = tweet['user']['id']
    df_tratado['user_id_str'] = tweet['user']['id_str']
    df_tratado['user_screen_name'] = tweet['user']['screen_name']
    df_tratado['user_location'] = tweet['user']['location']
    df_tratado['user_description'] = tweet['user']['description']
    df_tratado['user_protected'] = tweet['user']['protected']
    df_tratado['user_verified'] = tweet['user']['verified']
    df_tratado['user_followers_count'] = tweet['user']['followers_count']
    df_tratado['user_friends_count'] = tweet['user']['friends_count']

    # copy(): to avoid using the pointer concept, changes made to the copied object will not affect the     original object. If we do not use copy(), making only a simple assignment, the changes made to the     new object will also be reflected in the original object.
    user_mentions = []
    for i in range(len(tweet['entities']['user_mentions'])):
        dict_base = tweet['entities']['user_mentions'][i].copy()
        dict_base.pop('indices', None)
        df = pd.DataFrame(dict_base, index=[0])
        df = df.rename(columns={
            'screen_name':'entities_screen_name'
            ,'name':'entities_name'
            ,'id':'entities_id'
            ,'id_str':'entities_id_str'
        })
        user_mentions.append(df)

    dfs = []
    for i in user_mentions:
        dfs.append(
            pd.concat([df_tratado.copy(), i], axis=1)
        )

    df_final = pd.concat(dfs, ignore_index=True)
    return df_final

In [6]:
tweet_to_df(parsed_tweets[0])

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.